# $sa$-rectangular RMDPでの方策勾配について

* [First-order Policy Optimization for Robust Markov Decision Process](https://arxiv.org/abs/2209.10579)
    * Remark 1.1でなぜ遷移のUncertainty setを考えるのか？みたいな話をしていて良い

MDP $\mathcal{M}_u$での価値を次で定義します：
$$
V_u^\pi(s)=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t c\left(s_t, a_t\right) \mid s_0=s, a_t \sim \pi\left(\cdot \mid s_t\right), s_{t+1} \sim \mathbb{P}_u\left(\cdot \mid s_t, a_t\right)\right], \quad \forall s \in \mathcal{S} .
$$

このとき，ロバストMDPでの最適方策は次で定義されます：
$$
\min \left\{V_r^\pi(s):=\max _{u \in \mathcal{U}} V_u^\pi(s): \pi \in \Pi\right\}
$$

## SA-rectangular set

以降では，Uncertainty setが$(s, a)$-rectangularであるとします．

$$
\mathbb{P}_u(\cdot \mid s, a)=\mathbb{P}_{\mathbb{N}}(\cdot \mid s, a)+u(\cdot \mid s, a), \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$

ここで，$\mathbb{P}_N(\cdot |s, a)$はノミナルモデル，$u \in \mathbb{R}^{|\mathcal{S}| \times|\mathcal{S}| \times|\mathcal{A}|}$はノイズです．
$\mathbb{P}_N$をもつMDPをノミナルMDPと呼ぶことにします．
$\mathcal{U}$が次を満たすとき，このUncertainty setを$sa$-rectangularと言います：

$$
\mathcal{U}=\Pi_{s \in \mathcal{S}, a \in \mathcal{A}} \mathcal{U}_{s, a} \text {, where } \mathcal{U}_{s, a}=\{u(\cdot \mid s, a): u \in \mathcal{U}\} \text {. }
$$

以降，$\mathcal{U}$はコンパクトであるとします．
コンパクトな$\mathcal{U}$については次のBellman方程式が成立します：

$$
V_r^\pi(s)=\sum_{a \in \mathcal{A}} c(s, a) \pi(a \mid s)+\gamma \sum_{a \in \mathcal{A}} \pi(a \mid s) \max _{u \in \mathcal{U}} \sum_{s^{\prime} \in \mathcal{S}} \mathbb{P}_u\left(s^{\prime} \mid s, a\right) V_r^\pi\left(s^{\prime}\right), \quad \forall s \in \mathcal{S} .
$$

$$
Q_r^\pi(s, a)=c(s, a)+\max _{u \in \mathcal{U}} \gamma \sum_{s^{\prime} \in \mathcal{S}} \mathbb{P}_u\left(s^{\prime} \mid s, a\right) \sum_{a^{\prime} \in \mathcal{A}} \pi\left(a^{\prime} \mid s^{\prime}\right) Q_r^\pi\left(s^{\prime}, a^{\prime}\right), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A} .
$$


最後に，最悪ケースの遷移モデルを次で定義します：
$$
u_\pi(\cdot \mid s, a) \in \underset{u(\mid s, a) \in \mathcal{U}_{s, a}}{\operatorname{Argmax}} \sum_{s^{\prime} \in \mathcal{S}} \mathbb{P}_u\left(s^{\prime} \mid s, a\right) V_r^\pi\left(s^{\prime}\right), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$


## ロバストMDPと微分可能性について

ロバストMDPの最適方策を，次の目的関数を通じて求めることを考えましょう：
$$
\min _{\pi \in \Pi}\left\{f_\rho(\pi):=\mathbb{E}_{s \sim \rho} V_r^\pi(s)\right\},
$$
単純に考えるとこの問題を方策勾配法で解けば最適解が得られそうですが，そもそもこの目的関数における勾配は定義できるのでしょうか？
ロバストではないケースでも方策の勾配の定義には注意が必要です．

価値関数は確率方策の集合$\Pi$上に定義されているため，$\mathbf{1}^{\top} \pi(\cdot \mid s)=1$ for all $s \in \mathcal{S}$が成立していないといけません．よって，$f(\rho)$の定義域$\Pi=\operatorname{dom}\left(f_\rho\right)$は$\mathbb{R}^{|\mathcal{S}| \times|\mathcal{A}|}$の部分集合であり，その次元は$(|\mathcal{A}|-1) \mid \mathcal{S}$です．

よって，通常の勾配$\nabla f_\rho$は次を満たすべきですが，

$$
\lim _{\delta \rightarrow \mathbf{0}, \delta \in \mathbb{R}^{|\mathcal{S}| \times|\mathcal{A}|}} \frac{\mid f_\rho(\pi+\delta)-f_\rho(\pi)-\left\langle\nabla f_\rho(\pi), \delta\right\rangle \mid}{\|\delta\|} \rightarrow 0
$$

この$\delta$で摂動した結果が$\Pi$に入らないことがあるので，これはwell-definedではありません．
その代わりに，次の勾配の定義を考えましょう．また，ここではdirect parameterizationについて考えます．

---

**定義: 方策勾配**

任意の関数$f : \Pi \to \mathbb{R}$について，方策勾配$\nabla f(\pi)$を次で定義します：
$$
\lim _{\delta \rightarrow 0, \pi+\delta \in \Pi}
\frac{|f(\pi+\delta)-f(\pi)-\langle\nabla f(\pi), \delta\rangle|}{\|\delta\|_2} \rightarrow 0 .
$$

このとき，$\nabla f(\pi)$が存在するならばそれはuniqueであることがわかります．ちなみにこれはフレシェ微分を一般化しているっぽいです（TODO: フレシェ微分についてまとめよう）．

---

**補題：方策勾配**

固定された$u \in \mathcal{U}$に対して，方策勾配は次で与えられます：
よく知られているように，$V_u^\pi(s)$における方策勾配は
$$
\nabla V_u^\pi(s)\left[s^{\prime}, a\right]=\frac{1}{1-\gamma} d_s^{\pi, u}\left(s^{\prime}\right) Q_u^\pi\left(s^{\prime}, a\right), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$
で与えられます．
ここで，$\nabla V_u^\pi(s)\left[s^{\prime}, a\right]$は$\nabla V_u^\pi(s)$の$(s', a)$番目の要素の値です．

**証明**

Performance difference lemmaより，
任意の$\mathcal{M}_u$と任意の$\pi^{\prime}=\pi+\delta$に対して，

$$
\begin{aligned}
V_u^{\pi^{\prime}}(s)-V_u^\pi(s) & \stackrel{(a)}{=} \frac{1}{1-\gamma} \mathbb{E}_{s^{\prime} \sim d_s^{\pi^{\prime}, u}}\left\langle Q_u^\pi\left(s^{\prime}, \cdot\right), \pi^{\prime}\left(\cdot \mid s^{\prime}\right)-\pi\left(\cdot \mid s^{\prime}\right)\right\rangle \\
& =\frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi^{\prime}, u}\left(s^{\prime}\right) Q_u^\pi\left(s^{\prime}, a\right)\left(\pi^{\prime}\left(a \mid s^{\prime}\right)-\pi\left(a \mid s^{\prime}\right)\right) \\
& =\frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi, u}\left(s^{\prime}\right) Q_u^\pi\left(s^{\prime}, a\right)\left(\pi^{\prime}\left(a \mid s^{\prime}\right)-\pi\left(a \mid s^{\prime}\right)\right)\\
& +\frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}}\left(d_s^{\pi^{\prime}, u}\left(s^{\prime}\right)-d_s^{\pi, u}\left(s^{\prime}\right)\right) Q_u^\pi\left(s^{\prime}, a\right)\left(\pi^{\prime}\left(a \mid s^{\prime}\right)-\pi\left(a \mid s^{\prime}\right)\right) \\
& =\underbrace{\frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi, u}\left(s^{\prime}\right) Q_u^\pi\left(s^{\prime}, a\right) \delta\left(a \mid s^{\prime}\right)}_{(A)} \\
& +\underbrace{\frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}}\left(d_s^{\pi^{\prime}, u}\left(s^{\prime}\right)-d_s^{\pi, u}\left(s^{\prime}\right)\right) Q_u^\pi\left(s^{\prime}, a\right)\left(\pi^{\prime}\left(a \mid s^{\prime}\right)-\pi\left(a \mid s^{\prime}\right)\right)}_{(B)}, \\
&
\end{aligned}
$$
が成り立ちます．ここで$(A)=\langle g, \delta \rangle$とすると，$g\left(a \mid s^{\prime}\right)=d_s^{\pi, u}\left(s^{\prime}\right) Q_u^\pi\left(s^{\prime}, a\right)$に対応します．

これがちゃんと方策勾配の定義に一致していることを示すには，
$$(B)=\mathcal{o}\left(\left\|\pi-\pi^{\prime}\right\|_2\right)$$
であることを示せば十分です．$\sum_a$以降に$\pi'-\pi$の項があるので，結局
$$\left|d_s^{\pi^{\prime}, u}\left(s^{\prime}\right)-d_s^{\pi, u}\left(s^{\prime}\right)\right|=\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right)$$
を示せばよいです．

まずOccupancy measureについて
$$
\begin{aligned}
d_s^{\pi^{\prime}, u}-d_s^{\pi, u} & =(1-\gamma)\left(\left(I-\gamma \mathbb{P}_u^{\pi^{\prime}}\right)^{-1}-\left(I-\gamma \mathbb{P}_u^\pi\right)^{-1}\right) e_s \\
& =(1-\gamma) \gamma\left(I-\gamma \mathbb{P}_u^{\pi^{\prime}}\right)^{-1}\left(\mathbb{P}_u^\pi-\mathbb{P}_u^{\pi^{\prime}}\right)\left(I-\gamma \mathbb{P}_u^\pi\right)^{-1} e_s
\end{aligned}
$$
であることがすぐにわかります．ここでは$A^{-1}-B^{-1}=A^{-1}(B-A) B^{-1}$を使いました．すると，

$$\left\|\left(I-\gamma \mathbb{P}_u^\pi\right)^{-1}\right\|_1=\left(\min \left\{\left\|\left(I-\gamma \mathbb{P}_u^\pi\right) x\right\|_1:\|x\|_1=1\right\}\right)^{-1} \leq(1-\gamma)^{-1}$$
なので，

$$
\left\|d_s^{\pi^{\prime}, u}-d_s^{\pi, u}\right\|_1 \leq \frac{\gamma}{1-\gamma}\left\|\mathbb{P}_u^{\pi^{\prime}}-\mathbb{P}_u^\pi\right\|_1 \stackrel{(a)}{=} \mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_{\infty}\right) \stackrel{(b)}{=} \mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right),
$$

が成立します．
ここで(a)は$\left\|\pi-\pi^{\prime}\right\|_{\infty}$をS,Aなどで定数倍すれば抑えられること，そして(b)は$\|x\|_{\infty} \leq\|x\|_2 \leq \sqrt{n}\|x\|_{\infty}$を使ってます．
これで証明終わりです．

---

以上を踏まえると，ロバストMDPにおけるフレシェ劣微分が得られます．すなわち，$f_\rho$をdomainが$\Pi$な実数値関数とみなすわけですね．
フレシェ微分については[OPT_gradient.ipynb](OPT_gradient.ipynb)を参照してください．

---

$\bar{f}_\rho: \mathbb{R}^{|\mathcal{S}||\mathcal{A}|} \rightarrow \bar{R}=\mathbb{R} \cup\{+\infty\}$
を$f_\rho$の拡張関数とします．つまり，
$\bar{f}_\rho(\pi)=f_\rho(\pi)$ for any $\pi \in \Pi$ and $\bar{f}_\rho(\pi)=\infty$です．

ここで，
$$
\nabla f_\rho(\pi)[s, a]=\frac{1}{1-\gamma} d_\rho^{\pi, u_\pi}(s) Q_{u_\pi}^\pi(s, a), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A}
$$
を定義しましょう．このとき，$\nabla f_\rho(\pi)$は$\bar{f}_\rho$の$\pi$でのフレシェ劣微分です．

**証明**

$\pi, \pi' \in \Pi$について，$\delta=\pi'-\pi$とします．
このとき，
$$
\begin{aligned}
 V_{u_{\pi^{\prime}}}^{\pi^{\prime}}(s)-V_{u_\pi}^\pi(s)&=V_{u_{\pi^{\prime}}}^{\pi^{\prime}}(s)-V_{u_\pi}^{\pi^{\prime}}(s)+V_{u_\pi}^{\pi^{\prime}}(s)-V_{u_\pi}^\pi(s)\\
& \stackrel{(a)}{\geq} V_{u_\pi}^{\pi^{\prime}}(s)-V_{u_\pi}^\pi(s) \\
& \stackrel{(b)}{=} \frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi, u_\pi}\left(s^{\prime}\right) Q_{u_\pi}^\pi\left(s^{\prime}, a\right) \delta\left(a \mid s^{\prime}\right)+\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right),
\end{aligned}
$$

が成立します．ここで，(a)は$u_\pi$の定義より成立，(b)は多分Performance difference lemma的なことをしたら成立します（論文参照）．
よって，$f_\rho$の定義より，

$$
\begin{aligned}
f_\rho\left(\pi^{\prime}\right)-f_\rho(\pi) & \geq \frac{1}{1-\gamma} \sum_{s \in \mathcal{S}} \rho(s) \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi, u_\pi}\left(s^{\prime}\right) Q_{u_\pi}^\pi\left(s^{\prime}, a\right) \delta\left(a \mid s^{\prime}\right)+\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right) \\
& =\frac{1}{1-\gamma} \sum_{s \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_\rho^{\pi, u_\pi}(s) Q_{u_\pi}^\pi(s, a) \delta(a \mid s)+\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right),
\end{aligned}
$$

両辺を$\|\delta\|=\|\pi'-\pi\|$で割り，さらに$\pi'\neq \pi$のinfimumを取り，そして$\|\delta\| \downarrow 0$をすれば，

$$
\liminf _{\pi^{\prime} \rightarrow \pi, \pi^{\prime} \neq \pi}\left(f_\rho\left(\pi^{\prime}\right)-f_\rho(\pi)-\frac{1}{1-\gamma} \sum_{s \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_\rho^{\pi, u_\pi}(s) Q_{u_\pi}^\pi(s, a) \delta(a \mid s)\right) /\left\|\pi^{\prime}-\pi\right\|_2 \geq 0
$$

が成立するので，ちゃんとフレシェ劣微分になっています．

---


TODO: 至るところ微分可能の証明．

## Robust Policy Mirror Descent

次の更新を考えましょう．

$$
\pi_{k+1}(\cdot \mid s)=\underset{p(\cdot \mid s) \in \Delta_{\mathcal{A}}}{\operatorname{argmin}} \eta_k\left\langle Q_r^{\pi_k}(s, \cdot), p(\cdot \mid s)\right\rangle+D_{\pi_k}^p(s), \forall s \in \mathcal{S} .
$$

ここで$D$はBregman divergenceですが，一般にKLダイバージェンスを考えます．

$$
D_{\pi^{\prime}}^\pi(s)=\sum_{a \in \mathcal{A}} \pi(a \mid s) \log \left(\pi(a \mid s) / \pi^{\prime}(a \mid s)\right):=\operatorname{KL}\left(\pi(\cdot \mid s) \| \pi^{\prime}(\cdot \mid s)\right) .
$$

また，$D_w=\max _{\pi \in \Pi} \max _{s \in \mathcal{S}} D_{\pi_0}^\pi(s)$とします．

この更新の収束レートを見てみましょう．まず次が成立します．

---

任意の$p \in \Delta_\mathcal{A}$と$s \in \mathcal{S}$について，

$$
\eta_k\left\langle Q_r^{\pi_k}(s, \cdot), \pi_{k+1}(\cdot \mid s)-p\right\rangle \leq D_{\pi_k}^p(s)-D_{\pi_{k+1}}^p(s) - D_{\pi_k}^{\pi_{k+1}}(s)
$$

が成立します．

**証明**

論文中だとBregman divergenceの定義から示してますが，Mirror descentのThree point descent lemmaから一発で言えます．

---

続いて次が成立します：

$$
f_\rho\left(\pi_{k+1}\right)-f_\rho\left(\pi^*\right) \leq\left(1-\frac{1-\gamma}{M}\right)\left(f_\rho\left(\pi_k\right)-f_\rho\left(\pi^*\right)\right)+\frac{1}{M \eta_k} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_k}} D_{\pi_k}^{\pi^*}(s)-\frac{1}{M \eta_k} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_k}} D_{\pi_{k+1}}^{\pi^*}(s),
$$

ここで$M:=\sup _{u \in \mathcal{U}}\left\|d_\rho^{\pi^*, u} / \rho\right\|_{\infty}$としました．

**証明**

上で示したThree point descent lemmaから，$p=\pi_k$を代入すると

$$
\eta_k\left\langle Q_r^{\pi_k}(s, \cdot), \pi_{k+1}(\cdot \mid s)-\pi_k(\cdot \mid s)\right\rangle \leq-D_{\pi_{k+1}}^{\pi_k}(s)-D_{\pi_k}^{\pi_{k+1}}(s) \leq 0, \forall s \in \mathcal{S} .
$$

です．
一方で，$p=\pi^\star$を代入すると

$$
\underbrace{\eta_k\left\langle Q_r^{\pi_k}(s, \cdot), \pi_{k+1}(\cdot \mid s)-\pi_k(\cdot \mid s)\right\rangle}_{(A)}+\underbrace{\eta_k\left\langle Q_r^{\pi_k}(s, \cdot), \pi_k(\cdot \mid s)-\pi^*(\cdot \mid s)\right\rangle}_{(B)}+D_{\pi_k}^{\pi_{k+1}}(s) \leq D_{\pi_k}^{\pi_k^*}(s)-D_{\pi_{k+1}}^{\pi^*}(s) .
$$

です．
以降では$u_k=u_{\pi_k}$を方策$\pi_k$のもとでの最悪ケースのuncertaintyとします．
まず（A）の項について，

$$
\begin{aligned}
V_r^{\pi_{k+1}}(s)-V_r^{\pi_k}(s) & \stackrel{(a)}{\leq} \frac{1}{1-\gamma} \mathbb{E}_{s^{\prime} \sim d_s^{\pi_{k+1}, u_{k+1}}}\left\langle Q_r^{\pi_k}, \pi_{k+1}-\pi_k\right\rangle_{s^{\prime}} \\
& \stackrel{(b)}{\leq} \frac{d_s^{\pi_{k+1}, u_{k+1}}(s)}{1-\gamma}\left\langle Q_r^{\pi_k}(s, \cdot), \pi_{k+1}(\cdot \mid s)-\pi_k(\cdot \mid s)\right\rangle \\
& \stackrel{(c)}{\leq}\left\langle Q_r^{\pi_k}(s, \cdot), \pi_{k+1}(\cdot \mid s)-\pi_k(\cdot \mid s)\right\rangle \leq 0
\end{aligned}
$$

が成立します．(a)はPerformance difference lemma，(b)は各$s$で$0$以下であることを使い，(c)は$d_s^{\pi_{k+1}, u_{k+1}}(s) \geq(1-\gamma)$ for all $s \in \mathcal{S}$のせいです．

続いて(B)の項を変形しましょう．まず次が成立します（Performance difference lemmaに似てますが，ここでは不等式の形で成立します）：

$$
\mathbb{E}_{s^{\prime} \sim d_s^{\pi^*, u_k}}\left[\left\langle Q_r^{\pi_k}, \pi_k-\pi^*\right\rangle_{s^{\prime}}\right] \geq(1-\gamma)\left(V_r^\pi(s)-V_r^{\pi^*}(s)\right) \geq 0
$$

証明は論文のLemma 2.7に任せます．以上の結果をAとBに代入すると

$$
\begin{aligned}
& \mathbb{E}_{s^{\prime} \sim d_s^{\pi^*, u_k}}\left[\eta_k\left(V_r^{\pi_{k+1}}\left(s^{\prime}\right)-V_r^{\pi_k}\left(s^{\prime}\right)\right)\right]+(1-\gamma) \eta_k\left(V_r^{\pi_k}(s)-V_r^{\pi^*}(s)\right) \\
& \leq \mathbb{E}_{s^{\prime} \sim d_s^{\pi^*, u_k}} D_{\pi_k}^{\pi^*}\left(s^{\prime}\right)-\mathbb{E}_{s^{\prime} \sim d_s^{\pi^*, u_k}} D_{\pi_{k+1}}^{\pi^*}\left(s^{\prime}\right) \\
&
\end{aligned}
$$

最後に$s \sim \rho$で期待値を取ると，

$$
M\left[f_\rho\left(\pi_{k+1}\right)-f_\rho\left(\pi_k\right)\right]+(1-\gamma)\left[f_\rho\left(\pi_k\right)-f_\rho\left(x^*\right)\right] \leq \frac{1}{\eta_k} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_k}} D_{\pi_k}^{\pi^*}(s)-\frac{1}{\eta_k} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_k}} D_{\pi_{k+1}}^{\pi^*}(s),
$$

が成立します．

さて，この不等式の両辺で和を取ると，
$$
\begin{aligned}
& f_\rho\left(\pi_k\right)-f_\rho\left(\pi^*\right)+\frac{1-\gamma}{M} \sum_{t=1}^{k-1}\left(f_\rho\left(\pi_t\right)-f_\rho\left(\pi^*\right)\right) \\
\leq & f_\rho\left(\pi_0\right)-f_\rho\left(\pi^*\right)+\sum_{t=0}^{k-1}\left(\frac{1}{M \eta} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_t}} D_{\pi_t}^{\pi^*}(s)-\frac{1}{M \eta} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_t}} D_{\pi_{t+1}}^{\pi^*}(s)\right)
\end{aligned}
$$
が成り立ちます．

特に$w(\cdot)=\|\cdot\|_2^2$の場合，
$$
D_{\pi_t}^{\pi^*}(s)-D_{\pi_{t+1}}^{\pi^*}(s) \leq \sqrt{18 D_{\pi_{t+1}}^{\pi_t}(s)},
$$
であり，
$$
\sum_i\left(a_i-b_i\right)^2-\sum_i\left(a_i-c_i\right)^2 \leq\|b+c-2 a\|_2\|b-c\|_2 \leq \sqrt{18}\|b-c\|_2 .
$$
を使うと，
$$
f_\rho\left(\pi_k\right)-f_\rho\left(\pi^*\right)+\frac{1-\gamma}{M} \sum_{t=1}^{k-1}\left(f_\rho\left(\pi_t\right)-f_\rho\left(\pi^*\right)\right) \leq f_\rho\left(\pi_0\right)-f_\rho\left(\pi^*\right)+\frac{\sqrt{18}}{M \eta} \sum_{t=0}^{k-1} \mathbb{E}_{s \sim d_\rho^{\pi^*, u_t}} \sqrt{D_{\pi_{t+1}}^{\pi_t}(s)}
$$
が成立します．
よって，
$$
\left(\sum_{t=0}^{k-1} \sqrt{D_{\pi_{t+1}}^{\pi_t}(s)}\right)^2 \leq k \sum_{t=0}^{k-1} D_{\pi_{t+1}}^{\pi_t}(s) \leq k \eta\left(V_r^{\pi_0}(s)-V_r^{\pi^*}(s)\right) \leq \frac{k \eta}{1-\gamma}, \quad \forall s \in \mathcal{S},
$$
なので，

$$
f_\rho\left(\pi_k\right)-f_\rho\left(\pi^*\right)+\frac{1-\gamma}{M} \sum_{t=1}^{k-1}\left(f_\rho\left(\pi_t\right)-f_\rho\left(\pi^*\right)\right) \leq f_\rho\left(\pi_0\right)-f_\rho\left(\pi^*\right)+\frac{|\mathcal{S}|}{M} \sqrt{\frac{18 k}{\eta(1-\gamma)}} .
$$
が成り立ちます．

## 変分不等式を使った定式化

まず，次が成立します：

$$
V_r^{\pi^{\prime}}(s)-V_r^\pi(s) \leq \frac{1}{1-\gamma} \mathbb{E}_{s^{\prime} \sim d_s^{\pi^{\prime}, u_{\pi^{\prime}}}}\left\langle Q_r^\pi, \pi^{\prime}-\pi\right\rangle_{s^{\prime}} .
$$

証明は省略．これより，
$$
0 \leq V_r^\pi(s)-V_r^{\pi^*}(s) \leq \frac{1}{1-\gamma} \mathbb{E}_{s^{\prime} \sim d_s^{\pi, u \pi}}\left\langle Q_r^{\pi^*}, \pi-\pi^*\right\rangle_s=\left\langle\mathcal{F}_s\left(\pi, \pi^*\right), \pi-\pi^*\right\rangle,
$$
が成立します．ここで，$\mathcal{F}_s\left(\pi, \pi^{\prime}\right)\left[s^{\prime}, a\right]=\frac{1}{1-\gamma} d_s^{\pi, u_\pi}\left(s^{\prime}\right) Q_r^{\pi^{\prime}}\left(s^{\prime}, a\right) \quad \forall\left(s^{\prime}, a\right) \in \mathcal{S} \times \mathcal{A}$としました．

$\pi^*$の最適性より，$\left\langle\mathcal{F}\left(\pi, \pi^*\right), \pi-\pi^*\right\rangle \geq 0$が任意の$\pi \in \Pi$で成立します．ここで，$\pi_\alpha=\alpha \pi+(1-\alpha) \pi^*$とすると，

$$
\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi_\alpha-\pi^*\right\rangle=\alpha\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi-\pi^*\right\rangle \geq 0
$$

なので，

$$
\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi-\pi^*\right\rangle \geq 0
$$

です．ごにょごにょすると（TODO: ），
$$
\begin{aligned}
\lim _{\alpha \downarrow 0}\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi-\pi^*\right\rangle \geq 0 & \Rightarrow\left\langle\mathcal{G}_s\left(\pi^*\right), \pi-\pi^*\right\rangle \geq 0 \\
\mathcal{G}_s\left(\pi^*\right)\left[s^{\prime}, a\right] & =\frac{1}{1-\gamma} d_s^{\pi^*, u_{\pi^*}}\left(s^{\prime}\right) Q_r^{\pi^*}\left(s^{\prime}, a\right), \quad \forall\left(s^{\prime}, a\right) \in \mathcal{S} \times \mathcal{A}
\end{aligned}
$$
であることがわかります．よって，最適方策は次の変分不等式を解けば得られるわけですね：

$$
\left\langle\mathbb{E}_{s \sim \rho}\left[\mathcal{G}_s\left(\pi^*\right)\right], \pi-\pi^*\right\rangle \geq 0, \quad \forall \pi \in \Pi .
$$

ちなみにロバストではないMDPの最適方策は
$$
\begin{aligned}
&\left\langle\mathbb{E}_{s \sim \nu^*}\left[\mathcal{G}_s^N\left(\pi^*\right)\right], \pi-\pi^*\right\rangle \geq 0,\\
&\mathcal{G}_s^N\left(\pi^*\right)\left[s^{\prime}, a\right]=\frac{1}{1-\gamma} d_s^{\pi^*}\left(s^{\prime}\right) Q^{\pi^*}\left(s^{\prime}, a\right), \quad \forall\left(s^{\prime}, a\right) \in \mathcal{S} \times \mathcal{A},
\end{aligned}
$$

で求まります（TODO: [Policy Mirror Descent for Reinforcement Learning: Linear Convergence, New Sampling Complexity, and Generalized Problem Classes](https://arxiv.org/abs/2102.00135)を読め）．ここで$\nu^*$は最適方策の定常状態分布です．つまり，Q関数をロバストにしただけですね．

通常のMDPの場合はPerformance difference lemmaを利用すると
$$
\left\langle\mathbb{E}_{s \sim \nu^*}\left[\mathcal{G}_s^N(\pi)\right], \pi-\pi^*\right\rangle=\mathbb{E}_{s \sim \nu^*}\left[V^\pi(s)-V^{\pi^*}(s)\right]=f_{\nu^*}(\pi)-f_{\nu^*}\left(\pi^*\right)>0,
$$
が成立し，これを利用して変分不等式を解くことができます．しかし，実はロバストMDPではこれが成立しません（Proposition 2.3参照）．


